In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from pymongo import MongoClient
import time

client = MongoClient("mongodb://localhost:27017")
db = client["test0912"]
collection = db["yeogi"]

service = Service(ChromeDriverManager().install())

options = Options()
options.add_argument("--window-size=1920x1080")
options.add_argument("--start-maximized")
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36")
options.add_argument("--lang=ko_KR")

driver = webdriver.Chrome(service=service, options=options)

url = "https://www.yeogi.com/domestic-accommodations?keyword=%EA%B0%95%EB%A6%89&checkIn=2025-09-11&checkOut=2025-09-12&personal=2&freeForm=false&page=1"
driver.get(url)

wait = WebDriverWait(driver, 10)

for page_num in range(1, 6):
    print(f"\n===== {page_num} 페이지 데이터 =====")
    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    urls = driver.find_elements(By.CSS_SELECTOR, "a.gc-thumbnail-type-seller-card")
    titles = driver.find_elements(By.CSS_SELECTOR, "div.css-1j7tt62 > div > h3")
    prices = driver.find_elements(By.CSS_SELECTOR, "span.css-5r5920")
    ratings = driver.find_elements(By.CSS_SELECTOR, "span.css-9ml4lz")
    places = driver.find_elements(By.CSS_SELECTOR, "span.css-ki0lqh")
    reviews = driver.find_elements(By.CSS_SELECTOR, "span.css-oj6onp")
    
    page_data = []
    
    for i in range(len(titles)):
        record = {
            "title": titles[i].text if i < len(titles) else None,
            "price": prices[i].text.replace(",", "") if i < len(prices) else None,
            "rating": ratings[i].text if i < len(ratings) else None,
            "place": places[i].text if i < len(places) else None,
            "review_count": reviews[i].text.replace("명 평가", "").replace(",", "") if i < len(reviews) else None,
            "url": urls[i].get_attribute("href") if i < len(urls) else None,
            "page": page_num
        }
        print(record)
        page_data.append(record)
    
    if page_data:
        collection.insert_many(page_data)
        print(f"👉 {len(page_data)}개 데이터 MongoDB 저장 완료!")

    if page_num == 5:
        break
    
    try:
        next_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[aria-label='다음']")))
        next_btn.click()
        time.sleep(2)
    except Exception as e:
        print("다음 버튼 클릭 실패:", e)
        break

driver.quit()
client.close()


===== 1 페이지 데이터 =====
{'title': '★당일특가★ 세인트존스 호텔', 'price': '161820', 'rating': '9.1', 'place': '강릉 강문해변 앞', 'review_count': '9919', 'url': 'https://www.yeogi.com/domestic-accommodations/48951?checkIn=2025-09-12&checkOut=2025-09-13&personal=2', 'page': 1}
{'title': '★당일특가★ 스카이베이호텔 경포', 'price': '209470', 'rating': '9.1', 'place': '강문해변 차량 2분', 'review_count': '6697', 'url': 'https://www.yeogi.com/domestic-accommodations/49688?checkIn=2025-09-12&checkOut=2025-09-13&personal=2', 'page': 1}
{'title': '정동진 썬크루즈호텔', 'price': '198000', 'rating': '9.1', 'place': '정동진역 차량 5분', 'review_count': '2998', 'url': 'https://www.yeogi.com/domestic-accommodations/67769?checkIn=2025-09-12&checkOut=2025-09-13&personal=2', 'page': 1}
{'title': 'SL 호텔 강릉', 'price': '105830', 'rating': '9.1', 'place': '주문진터미널 차량 4분', 'review_count': '2437', 'url': 'https://www.yeogi.com/domestic-accommodations/69744?checkIn=2025-09-12&checkOut=2025-09-13&personal=2', 'page': 1}
{'title': '강릉 로맨스 인 강문', 'price': '206692', 'r

In [16]:
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017")
db = client["test0912"]
collection = db["yeogi"]

pipeline = [
    {
        "$sort": {
            "rating": -1,
            "review_count": -1
        }
    },
    {
        "$limit": 10
    },
    {
        "$project": {
            "_id": 0,
            "title": 1,
            "rating": 1,
            "review_count": 1,
            "price": 1,
            "place": 1,
            "url": 1
        }
    }
]

results = list(collection.aggregate(pipeline))

print("\n=== 평점 상위 10 숙소(동률 시 리뷰 수 기준) ===")
for r in results:
    print(f"{r['title']} | 평점: {r['rating']} | 리뷰수: {r['review_count']} | 가격: {r['price']} | 위치: {r['place']} | URL: {r['url']}")

client.close()


=== 평점 상위 10 숙소 ===
강릉 라온 펜션 | 평점: 10.0 | 리뷰수: 176 | 가격: None | 위치: 강문해변 차량 3분 | URL: https://www.yeogi.com/domestic-accommodations/60202?checkIn=2025-09-12&checkOut=2025-09-13&personal=2
강릉 오릿 | 평점: 10.0 | 리뷰수: 122 | 가격: 79000 | 위치: 소돌해변 도보 10분 | URL: https://www.yeogi.com/domestic-accommodations/81827?checkIn=2025-09-12&checkOut=2025-09-13&personal=2
강릉 오리아뜰리에풀빌라펜션 | 평점: 10.0 | 리뷰수: 94 | 가격: 297000 | 위치: 강문해변 도보 2분 | URL: https://www.yeogi.com/domestic-accommodations/81245?checkIn=2025-09-12&checkOut=2025-09-13&personal=2
강릉 경포 서지별담 독채 스테이 | 평점: 10.0 | 리뷰수: 52 | 가격: 62100 | 위치: 경포호 차량 2분 | URL: https://www.yeogi.com/domestic-accommodations/82852?checkIn=2025-09-12&checkOut=2025-09-13&personal=2
강릉 스테이 이플 | 평점: 10.0 | 리뷰수: 40 | 가격: 260000 | 위치: 순포습지 도보 1분 | URL: https://www.yeogi.com/domestic-accommodations/76803?checkIn=2025-09-12&checkOut=2025-09-13&personal=2
강릉 피어하우스독채풀빌라 | 평점: 10.0 | 리뷰수: 14 | 가격: 30500 | 위치: 강문해변 차량 9분 | URL: https://www.yeogi.com/domestic-accommodations/83700?